In [1]:
# Find split of dataset
lbls_split = []
with open("datasetSplit.txt","r") as f:
    next(f)
    for line in f:
        [_, id_split] = line.strip().split(',')
        lbls_split.append(eval(id_split))
# Split dataset
words_train = []
sents_train = []
words_test = []
sents_test = []
with open("SOStr.txt","r") as f:
    for i, line in enumerate(f):
        if lbls_split[i] is 1:
            words_train.append(line.strip().split('|'))
            sents_train.append(line.strip().replace('|', ' '))
        elif lbls_split[i] is 2:
            words_test.append(line.strip().split('|'))
            sents_test.append(line.strip().replace('|', ' '))
# Extract word/sentency dictionary
dct = []
ids_dct = []
with open('dictionary.txt') as f:
    for line in f:
        line = line.strip()
        index = line.find('|')
        word, num = line.split('|')
        dct.append(word)
        ids_dct.append(eval(num))
# Extract ratings
ratings = []
with open('sentiment_labels.txt') as f:
    next(f)
    for line in f:
        [_, rate] = line.strip().split('|')
        ratings.append(eval(rate))
# Extract dataset labels
# Training
lbls_train = []
for sent in sents_train:
    rating = ratings[ids_dct[dct.index(sent)]]
    if rating >= 0 and rating < 0.2:
        lbls_train.append(0)
    elif rating >= 0.2 and rating < 0.4:
        lbls_train.append(1)
    elif rating >= 0.4 and rating < 0.6:
        lbls_train.append(2)
    elif rating >= 0.6 and rating < 0.8:
        lbls_train.append(3)
    elif rating >= 0.8 and rating <= 1:
        lbls_train.append(4)
# Testing
lbls_test = []
for sent in sents_test:
    rating = ratings[ids_dct[dct.index(sent)]]
    if rating >= 0 and rating < 0.2:
        lbls_test.append(0)
    elif rating >= 0.2 and rating < 0.4:
        lbls_test.append(1)
    elif rating >= 0.4 and rating < 0.6:
        lbls_test.append(2)
    elif rating >= 0.6 and rating < 0.8:
        lbls_test.append(3)
    elif rating >= 0.8 and rating <= 1:
        lbls_test.append(4)

In [2]:
# Smoothing variable
k = 1
# Calculate probabilities
Py = [0]*5
Pwgy = [dict(), dict(), dict(), dict(), dict()]
for w, words in enumerate(words_train):
    # Sentiment probability
    Py[lbls_train[w]] += 1
    for word in words:
        # Probability of word given sentiment
        if not word in Pwgy[lbls_train[w]]:
            Pwgy[lbls_train[w]][word] = k
        Pwgy[lbls_train[w]][word] += 1
for y in range(0, len(Py)):
    Py[y] /= len(sents_train)
# for sent in Pw:
#     Pw[sent] /= len(sents_train)
nwords = [k]*5
for w, words in enumerate(words_train):
    for word in words:
        nwords[lbls_train[w]] += 1
for y in range(0, len(Pwgy)):
    Pwgy[y][_] = k
    for word in Pwgy[y]:
        Pwgy[y][word] /= nwords[y]

In [3]:
print('Performance Using All Observed Words')
# Predict probabilities
Pygw = [0]*5
# Make predictions
# Training
corr = 0
for s, sent in enumerate(sents_train):
    # Probability of sentiment given sentence
    Pw = 0
    for y in range(0, len(Py)):
        Pwgy_prod = 1
        for word in words_train[s]:
            if word in Pwgy[y]:
                Pwgy_prod *= Pwgy[y][word]
            else:
                Pwgy_prod *= Pwgy[y][_]
        Pygw[y] = Py[y]*Pwgy_prod
        Pw += Pygw[y]
    for y in range(0, len(Py)):
        Pygw[y] /= Pw
    if Pygw.index(max(Pygw)) == lbls_train[s]:
        corr += 1
perf_train = corr/len(sents_train)*100
print('Training Performance: {0:.2f}%'.format(perf_train))
# Testing
corr = 0
for s, sent in enumerate(sents_test):
    # Probability of sentiment given sentence
    Pw = 0
    for y in range(0, len(Py)):
        Pwgy_prod = 1
        for word in words_test[s]:
            if word in Pwgy[y]:
                Pwgy_prod *= Pwgy[y][word]
            else:
                Pwgy_prod *= Pwgy[y][_]
        Pygw[y] = Py[y]*Pwgy_prod
        Pw += Pygw[y]
    for y in range(0, len(Py)):
        Pygw[y] /= Pw
    if Pygw.index(max(Pygw)) == lbls_test[s]:
        corr += 1
perf_test = corr/len(sents_test)*100
print('Testing Performance: {0:.2f}%'.format(perf_test))

Performance Using All Observed Words
Training Performance: 73.44%
Testing Performance: 33.17%


In [4]:
# Extract known positive and negative words
words_pos = []
with open('positive-words.txt', 'r', encoding = 'latin-1') as f:
    for i in range(0, 35):
        next(f)
    for line in f:
        words_pos.append(line.strip())
words_neg = []
with open('negative-words.txt', 'r', encoding = 'latin-1') as f:
    for i in range(0, 35):
        next(f)
    for line in f:
        words_neg.append(line.strip())
# Smoothing variable
k = 1
# Calculate probabilities
Py = [0]*5
Pwgy = [dict(), dict(), dict(), dict(), dict()]
for w, words in enumerate(words_train):
    # Sentiment probability
    Py[lbls_train[w]] += 1
    for word in words:
        # Probability of word given sentiment
        if word in words_pos or word in words_neg:
            if not word in Pwgy[lbls_train[w]]:
                Pwgy[lbls_train[w]][word] = k
            Pwgy[lbls_train[w]][word] += 1
for y in range(0, len(Py)):
    Py[y] /= len(sents_train)
# for sent in Pw:
#     Pw[sent] /= len(sents_train)
nwords = [k]*5
for w, words in enumerate(words_train):
    for word in words:
        if word in words_pos or word in words_neg:
            nwords[lbls_train[w]] += 1
for y in range(0, len(Pwgy)):
    Pwgy[y][_] = k
    for word in Pwgy[y]:
        Pwgy[y][word] /= nwords[y]

In [5]:
print('Performance Using Only Known Positive and Negative Words')
# Predict probabilities
Pygw = [0]*5
# Make predictions
# Training
corr = 0
for s, sent in enumerate(sents_train):
    # Probability of sentiment given sentence
    Pw = 0
    for y in range(0, len(Py)):
        Pwgy_prod = 1
        for word in words_train[s]:
            if word in Pwgy[y]:
                Pwgy_prod *= Pwgy[y][word]
            else:
                Pwgy_prod *= Pwgy[y][_]
        Pygw[y] = Py[y]*Pwgy_prod
        Pw += Pygw[y]
    for y in range(0, len(Py)):
        Pygw[y] /= Pw
    if Pygw.index(max(Pygw)) == lbls_train[s]:
        corr += 1
perf_train = corr/len(sents_train)*100
print('Training Performance: {0:.2f}%'.format(perf_train))
# Testing
corr = 0
for s, sent in enumerate(sents_test):
    # Probability of sentiment given sentence
    Pw = 0
    for y in range(0, len(Py)):
        Pwgy_prod = 1
        for word in words_test[s]:
            if word in Pwgy[y]:
                Pwgy_prod *= Pwgy[y][word]
            else:
                Pwgy_prod *= Pwgy[y][_]
        Pygw[y] = Py[y]*Pwgy_prod
        Pw += Pygw[y]
    for y in range(0, len(Py)):
        Pygw[y] /= Pw
    if Pygw.index(max(Pygw)) == lbls_test[s]:
        corr += 1
perf_test = corr/len(sents_test)*100
print('Testing Performance: {0:.2f}%'.format(perf_test))

Performance Using Only Known Positive and Negative Words
Training Performance: 14.96%
Testing Performance: 14.03%
